In [1]:
import pandas as pd

In [2]:
data_macrophage = pd.read_csv("../../app/macrophage_props_1dpi_and_5dpi.csv")

data_tumor_1dpi = pd.read_csv("../../app/tumor_volumes_and_distances_1dpi.csv")
data_tumor_5dpi = pd.read_csv("../../app/tumor_volumes_and_distances_5dpi.csv")
data_tumor_5dpi.columns

Index(['short_name', 'fish_id', 'time_point', 'tumor_volume', 'dt_min',
       'time_in_min', 'cancer_cells', 'dpi', 'PixelSizeX', 'PixelSizeY',
       'PixelSizeZ', 'time_in_h', 'tumor_volume_um',
       'macrophage_mean_distance'],
      dtype='object')

In [3]:
t_1dpi_start = 26, 27

In [4]:
limits_round = [0.6, 1.0]
limits_remified = [0.0, 0.35]

time_intervals = {  "t_1dpi_start": [24,26],
                    "t_1dpi_mid": [30,32],
                    "t_1dpi_end": [37,39],
                    "t_5dpi_start": [114,116],
                    "t_5dpi_end":  [117,119]}

t_1dpi_start = 24,26
t_1dpi_mid = 30,32
t_1dpi_end = 37,39

t_5dpi_start = 115
t_5dpi_end = 118

In [5]:
data_macrophage.columns

Index(['short_name', 'fish_id', 'time_point', 'number', 'Area', 'Mean', 'Min',
       'Max', 'X', 'Y', 'area_mum2', 'dt_min', 'time_in_min',
       'minor_axis_length_px', 'minor_axis_length_mum', 'major_axis_length_px',
       'major_axis_length_mum', 'major_minor_ratio', 'perimeter_px',
       'perimeter_mum', 'eccentricity', 'circularity', 'cancer_cells',
       't_start', 't_end', 'time_in_h', 'dpi'],
      dtype='object')

In [6]:
data_macrophage.groupby(["fish_id","time_point"]).count().std()[0]

24.19456030781344

In [7]:
data_round = data_macrophage[data_macrophage["circularity"] >= limits_round[0]]
data_remified = data_macrophage[data_macrophage["circularity"] <=  limits_remified[1]]

In [8]:
matrix_cols = ['feature'] + list(data_macrophage['cancer_cells'].unique())
#matrix_cols = matrix_cols.append(data.columns)
print(matrix_cols)
data_matrix =  pd.DataFrame(columns = matrix_cols)
data_matrix

['feature', 'LBT070', 'LBT123', 'LBT001', 'LBT003', 'BT333', 'CME037', 'CME038', 'BT569']


,feature,LBT070,LBT123,LBT001,LBT003,BT333,CME037,CME038,BT569


In [9]:
len(data_matrix .index)

0

In [10]:
def add_feature(data, data_matrix, index, feature, macrophage_type, time_interval):
    #index = len(data_cluster.index)
    
    t_start = time_intervals[time_interval][0]
    t_end = time_intervals[time_interval][1]
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "mean_%s_%s_%s" % (feature, macrophage_type,time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        if feature == "count":
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start.groupby(["fish_id","time_point"]).count().mean()[0]
        else:
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].mean()
        
    
    index += 1
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "std_%s_%s_%s" % (feature,macrophage_type,time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        if feature == "count":
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start.groupby(["fish_id","time_point"]).count().std()[0]
        else:
            data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].std()
    
    index += 1
      
    return data_matrix, index
    #index += 1
    

In [11]:
def add_feature_tumor(data, data_matrix, index, feature, time_interval):
    #index = len(data_cluster.index)
    
    t_start = time_intervals[time_interval][0]
    t_end = time_intervals[time_interval][1]
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "mean_%s_%s" % (feature, time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].mean()
        
    
    index += 1
    
    for cancer_cell in data['cancer_cells'].unique():
        data_cancer_cell = data[data['cancer_cells']==cancer_cell]
        data_matrix.at[index ,"feature"] = "std_%s_%s" % (feature,time_interval)
        data_cancer_cell_start = data_cancer_cell[data_cancer_cell['time_in_h'] >= t_start]
        data_cancer_cell_start =  data_cancer_cell_start[ data_cancer_cell_start['time_in_h'] <= t_end]
        data_matrix.at[index,cancer_cell] = data_cancer_cell_start[feature].std()
    
    index += 1
      
    return data_matrix, index
    #index += 1

In [12]:
index = 0
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "count", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "count", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "area_mum2", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "area_mum2", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "perimeter_mum", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "perimeter_mum", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "eccentricity", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "eccentricity", "remified", "t_5dpi_end")

data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_1dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_1dpi_mid")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_1dpi_end")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_5dpi_start")
data_matrix, index = add_feature(data_round, data_matrix, index, "circularity", "round", "t_5dpi_end")

data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_1dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_1dpi_mid")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_1dpi_end")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_5dpi_start")
data_matrix, index = add_feature(data_remified, data_matrix, index, "circularity", "remified", "t_5dpi_end")
data_matrix

,feature,LBT070,LBT123,LBT001,LBT003,BT333,CME037,CME038,BT569
0,mean_count_round_t_1dpi_start,17.972222,28.111111,13.166667,19.818182,19.866667,11.6,7.055556,18.638889
1,std_count_round_t_1dpi_start,7.12936,10.278447,8.463048,7.889446,5.237355,4.709492,2.689243,5.472804
2,mean_count_round_t_1dpi_mid,15.888889,24.844444,12.47619,18.363636,19.966667,12.033333,8.166667,21.0
3,std_count_round_t_1dpi_mid,5.086552,9.73269,5.03606,6.374556,8.442435,4.874164,2.915476,5.220837
4,mean_count_round_t_1dpi_end,18.305556,21.4,11.833333,17.292308,16.0,14.933333,10.444444,19.777778
...,...,...,...,...,...,...,...,...,...
95,std_circularity_remified_t_1dpi_end,0.07396,0.065372,0.055383,0.068799,0.051893,0.054782,0.060561,0.057231
96,mean_circularity_remified_t_5dpi_start,0.231879,0.229679,0.23763,0.242301,0.239467,0.246607,0.216012,0.221682
97,std_circularity_remified_t_5dpi_start,0.077757,0.076969,0.069436,0.077493,0.072274,0.076936,0.080046,0.086672
98,mean_circularity_remified_t_5dpi_end,0.230244,0.227336,0.2548,0.246279,0.245894,0.244623,0.228547,0.229521


# add tumor data

In [13]:
data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "tumor_volume_um", "t_1dpi_start")
data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "tumor_volume_um", "t_1dpi_mid")
data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "tumor_volume_um", "t_1dpi_end")
data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "tumor_volume_um", "t_5dpi_start")
data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "tumor_volume_um", "t_5dpi_end")

data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "macrophage_mean_distance", "t_1dpi_start")
data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "macrophage_mean_distance", "t_1dpi_mid")
data_matrix, index = add_feature_tumor(data_tumor_1dpi, data_matrix, index, "macrophage_mean_distance", "t_1dpi_end")
data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "macrophage_mean_distance", "t_5dpi_start")
data_matrix, index = add_feature_tumor(data_tumor_5dpi, data_matrix, index, "macrophage_mean_distance", "t_5dpi_end")
data_matrix

,feature,LBT070,LBT123,LBT001,LBT003,BT333,CME037,CME038,BT569
0,mean_count_round_t_1dpi_start,17.972222,28.111111,13.166667,19.818182,19.866667,11.6,7.055556,18.638889
1,std_count_round_t_1dpi_start,7.12936,10.278447,8.463048,7.889446,5.237355,4.709492,2.689243,5.472804
2,mean_count_round_t_1dpi_mid,15.888889,24.844444,12.47619,18.363636,19.966667,12.033333,8.166667,21.0
3,std_count_round_t_1dpi_mid,5.086552,9.73269,5.03606,6.374556,8.442435,4.874164,2.915476,5.220837
4,mean_count_round_t_1dpi_end,18.305556,21.4,11.833333,17.292308,16.0,14.933333,10.444444,19.777778
...,...,...,...,...,...,...,...,...,...
115,std_macrophage_mean_distance_t_1dpi_end,30.847888,25.949779,23.123578,31.815002,31.562488,22.065384,17.855857,26.809816
116,mean_macrophage_mean_distance_t_5dpi_start,120.297571,145.774625,129.195169,113.173526,78.349869,107.242757,158.061113,184.342916
117,std_macrophage_mean_distance_t_5dpi_start,46.867684,49.082846,14.746535,45.002447,27.497577,25.441836,21.094481,57.024883
118,mean_macrophage_mean_distance_t_5dpi_end,108.617336,160.13164,139.316206,110.871801,77.010152,112.704677,179.680079,188.563038


In [14]:
data_matrix.to_csv("tumor_macrophage_data_matrix_for_scRNA_seq.csv", index=False)